# Visualizacion varias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from collections import namedtuple
from scipy import stats
import seaborn as sns
import geopandas as gpd
import matplotlib
import ipywidgets as widgets
from ipywidgets import interact
matplotlib.style.use("ggplot")
import folium
from folium.plugins import DualMap 
from scripts import mapas, pygal_helpers, georef, download
from shapely.geometry import Polygon, MultiPolygon
from pygal_helpers import df_to_pygal_line, get_pygal_html
from datetime import datetime
from IPython.display import HTML, display

ImportError: dlopen(/Users/abenassi/anaconda/envs/proyecto-next-journal/lib/python3.7/site-packages/fiona/ogrext.cpython-37m-darwin.so, 2): Library not loaded: @rpath/libpoppler.76.dylib
  Referenced from: /Users/abenassi/anaconda/envs/proyecto-next-journal/lib/libgdal.20.dylib
  Reason: image not found

## Evolucion de genero en grupo de carreras

In [ ]:
df_evolucion_carreras = pd.read_csv('data/output/grupos_evolucion_genero.csv',delimiter=',')
df_evolucion_grupos_carreras = pd.read_csv('data/output/grupos_carreras_evolucion_genero.csv',delimiter=',')
df_evolucion_carreras.head(10)

In [ ]:
df_evolucion_carreras[
    (df_evolucion_carreras.carrera_grupo == "Derecho") &
    (df_evolucion_carreras.anio == 2012)
]

In [ ]:
#Comparacion de valores absolutos de hombres y mujeres - nuevas inscripciones/ filtro por carrera (eje primaro y secundario??)

In [ ]:
variables_carreras_evolucion = tuple(df_evolucion_carreras.carrera_grupo.unique())
widget_grupos=widgets.SelectMultiple(
    options=sorted(variables_carreras_evolucion),
    value=["Tecnologia"],
    #rows=10,
    description='Areas Tematicas',
    disabled=False
)

In [ ]:
#Version final
variables_indicadores_line = {
    "Ratio mujeres": "estudiantes_mujeres_pct",
    "Est. hombres": "estudiantes_hombres_pct"
}

widget_indicador = widgets.ToggleButtons(
    options=variables_indicadores_line.items(),
    description="Indicador : ",
    value="estudiantes_mujeres_pct"
)

def format_pct(value):
    if pd.notnull(value):
        return round((value*100),1)
    else:
        return value

@interact(carrera_grupo=widget_grupos, indicador=widget_indicador)
def generate_chart_evolucion_grupos(carrera_grupo, indicador):
    serie_grupo_carrera = df_evolucion_carreras[df_evolucion_carreras.carrera_grupo.isin(carrera_grupo)][[
        "anio", 'carrera_grupo',  indicador]]
    serie_grupo_carrera[indicador]=serie_grupo_carrera[indicador].apply(format_pct)
    df_pivot_serie_grupo_carrera = serie_grupo_carrera.pivot_table(
        index='anio', columns='carrera_grupo', values=indicador)
    return HTML(get_pygal_html(df_to_pygal_line(df_pivot_serie_grupo_carrera)))
        
#aclarar en nota que los valores son porcentajes

In [ ]:
#Version Carrera nombre
@interact(carrera_grupo=widget_grupos) #widget diferente
def generate_chart_evolucion_grupos(carrera_grupo):
    serie_grupo_carrera = df_evolucion_grupos_carreras[df_evolucion_grupos_carreras.carrera_grupo.isin(carrera_grupo)][[
        "anio", 'carrera_nombre',  "estudiantes_mujeres_pct"]]
    df_pivot_serie_grupo_carrera = serie_grupo_carrera.pivot_table(index='anio', columns='carrera_nombre', values='estudiantes_mujeres_pct')
    return HTML(get_pygal_html(df_to_pygal_line(df_pivot_serie_grupo_carrera)))
        

In [ ]:
#Una vez revisado, armar funcion para este tipo de grafico con un grupo de carrera, me devuelve todas las carreras.
# buscar una lib de grafico de linea con tooltip dinamico
#grupo = 'Economicas'
#df_grupo_carrera = df_evolucion_grupos_carreras[df_evolucion_grupos_carreras.carrera_grupo == grupo][[
#        "anio", "estudiantes_mujeres_pct", 'carrera_nombre', 'carrera_grupo'
#    ]]
#df_grupo_carrera.pivot_table(index='anio', columns='carrera_nombre', values='estudiantes_mujeres_pct').plot(
#figsize=(20,20))

In [ ]:
#variables_carreras_evolucion = df_evolucion_grupos_carreras.carrera_grupo.unique()
#said @interact(carrera_grupo = sorted(variables_carreras_evolucion))
#def generate_chart_evolucion_carreras(carrera_grupo):
#    df_grupo_carrera = df_evolucion_grupos_carreras[df_evolucion_grupos_carreras.carrera_grupo == carrera_grupo][[
#        "anio", "estudiantes_mujeres_pct"
#    ]]
#    return serie_grupo_carrera.set_index("anio").plot()
    
#agregar mas de un filtro para ir comparando
#modificar en ETL abogacia y derecho
#apertura por carreras para entender donde hace el pico y decrecimiento tech - cambiar pKEY por carrera y grupo carrera - grafico exploratorio
#segunda tabla de ETL donde haya grupo carrera (carrera) + universidades
#grafico con valores absolutos - apilados o algo que represente orden de magnitud año por default 

## Mapa universidades

In [ ]:
df_universidades = pd.read_csv('data/output/universidades_mapa.csv',delimiter=',')
df_evolucion_ubicacion = pd.read_csv('data/output/ubicacion_evolucion_genero.csv',delimiter=',', dtype={'provincia_id': str})
#revisar etl no perder el str de los numeros
#df_evolucion_ubicacion_test = pd.read_csv('data/output/ubicacion_evolucion_genero_test.csv',
                                          #delimiter=',', dtype={'provincia_id': str})
#revisar etl no perder el str de los numeros

In [ ]:
argentina_coords = [-40, -64]
osm_no_labels = "https://tiles.wmflabs.org/osm-no-labels/{z}/{x}/{y}.png"
ign_labels = "http://wms.ign.gob.ar/geoserver/gwc/service/tms/1.0.0/capabaseargenmap@EPSG%3A3857@png/{z}/{x}/{y}.png"

## Heat map

In [ ]:
df_evolucion_ubicacion['provincia_id'] = df_evolucion_ubicacion.provincia_id.str.zfill(2)

In [ ]:
def get_bins(min_value, max_value, bins_num=5):
    step = (max_value - min_value) / bins_num
    bins = list(np.arange(min_value, max_value, step))
    bins.append(max_value)
    return bins

In [ ]:
INDICADORES_HEATMAP = {
    "estudiantes_mujeres_pct" : "Ratio de estudiantes mujeres",
    "estudiantes_grupo_pct" : "Ratio de estudiantes",
    "estudiantes_varones" : "Total de estudiantes hombres",
    #"estudiantes_mujeres" : "Total de estudiantes mujeres", 
    #"inscriptos_mujeres_pct" : "Ratio de inscriptas mujeres ",
    #"inscriptos_hombres_pct" : "Ratio de inscriptas hombres ",
    #"estudiantes_hombres_pct" : "Ratio de estudiantes hombres ",
}
variables_indicadores = {
    "Ratio mujeres": "estudiantes_mujeres_pct",
    "Ratio grupo": "estudiantes_grupo_pct",
    "Est. hombres": "estudiantes_varones"
}

variables_carreras_heatmap = sorted(df_evolucion_ubicacion.carrera_grupo.unique())
variables_anio_heatmap = sorted(df_evolucion_ubicacion.anio.unique())

widget_indicador = widgets.ToggleButtons(
    options=variables_indicadores.items(),
    description="Indicador : ",
    value="estudiantes_mujeres_pct"
)

widget_carreras = widgets.Dropdown(
    options=variables_carreras_heatmap,
    description="Grupo : ",
    value="Tecnologia"
)

widget_anio = widgets.Dropdown(
    options=variables_anio_heatmap,
    description="Año : ",
    value=2016
)

widget_scale = widgets.RadioButtons(
    options=['Cuantiles', 'Fija', 'Relativa'],
    description='Escala :',
    value='Relativa'
)

@interact(anio=widget_anio,
          carrera_grupo=widget_carreras,
          indicador=widget_indicador,
          scale_type=widget_scale
         )
def generate_heat_map(anio, carrera_grupo, indicador, scale_type=None):
    mapa_heatmap = folium.Map(location=argentina_coords, zoom_start=4)
    folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa_heatmap)
    
    df_filter = df_evolucion_ubicacion[(df_evolucion_ubicacion.anio == anio)&
                           (df_evolucion_ubicacion.carrera_grupo == carrera_grupo)]
    
    if scale_type == 'Cuantiles':
        bins = list(df_filter[indicador].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
    elif scale_type == 'Fija':
        bins = get_bins(df_evolucion_ubicacion[indicador].min(), df_evolucion_ubicacion[indicador].max())
    else:
        bins = 5
    
    
    indicador_desc = INDICADORES_HEATMAP[indicador]
    folium.Choropleth(
        geo_data="data/input/provincias.geojson", # la capa de provincias en GeoJson
        data=df_filter, # DataFrame con los datos a graficar
        columns=['provincia_id', indicador], # columna con id de geometrías, columna con el dato
        key_on='feature.properties.id', # campo del geojson que tiene el id de las geometrías
        fill_color='YlGn', # escala de colores a usar
        #fill_opacity=0.7, # opacidad del color de relleno
        #line_opacity=0.2, # opacidad de las líneas que separan los polígonos
        legend_name='{} en {} (Año {})'.format(indicador_desc, carrera_grupo, anio), # título de la leyenda de la escala
        bins=bins
    ).add_to(mapa_heatmap)
    #mapas.add_labels_from_column(mapa_heatmap,df_evolucion_ubicacion,"nombre")

    return mapa_heatmap
#Agregar un tooltip que muestre informacion de la provincia al pasar el cursor. Nombre provincia, estudiantes totales, año, porcentaje de mujeres, cantidad de universidades por provincia.
#Hacer zoom sobre la cuidad de buenos aires. Hacer un grafico nuevo
#min, max y media de la variable analizada por grupo.
#Agregar grafico de evolucion y/y entre provincias por grupo de carreras. Grafico de barras con la composicion de los grupos
#Cambiar los nombres a los indices porque son muy tecnicos. Podriamos usar indice de feminidad, grupo??, masculidad
#Se puede cambiar la escala cuando no hay datos?? (blancos o transparente)
#Tabla con visualizacon y que tenga un orden de ranking para poder identificar facilmente cual es la provincia que mas avanzo

## Markers

In [ ]:
def mapear_universidades(mapa, df, lat_col, lon_col, institucion_nombre_col, carreras_grupos_col):
    """ cargar informacion al marker de forma masiva.

        Args:
            lat (float): latitud
            lon (float): longuitud
            institucion_nombre (str): nombre de la universidad
            carreras_grupos (list): lista con las areas de estudios ofrecidas

        """
    
    for row in df_universidades.iterrows():
        mapas.crear_marker(
            mapa,
            row[1]["lat"],
            row[1]["lon"],
            row[1]["institucion_nombre"],
            str(row[1]["carrera_grupo"]).split(",")
        )
#agregar feminidad por grupo carrera, elegir si queremos hacer selector por año o elegir 2016.    

In [ ]:
mapa_markers = folium.Map(location=argentina_coords, zoom_start=4)
folium.TileLayer(ign_labels, tms=True, attr="IGN").add_to(mapa_markers)

In [ ]:
mapear_universidades(mapa_markers,df_universidades,"lat","lon","institucion_nombre","carrera_grupo")

In [ ]:
mapa_markers
#Mapa + tabla "donde estudio" puedo hacer un filtro por grupo carrera (inverso a lo que estamos haciendo) 
#Cantidad de alumnas mujeres o comparacion de universidades
#agregar al tooltip el indice de feminidad